# Object Oriented Programming II

## Inheritance
Inheritance models an `is a` relationship. Meaning that the `Derived` class `is a` specialized version of the `Base` class which it inherits 

In [3]:
class Shape:
    def __init__(self,side):
        self.side = side
    def num_side(self):
        return self.side
    def get_area(self):
        raise NotImplementedError
    def get_perimeter(self):
        raise NotImplementedError
    

class Rectangle(Shape):
    def __init__(self,length,width):
        Shape.__init__(self,2)
        self.length = length
        self.width = width
    def get_area(self):
        return self.length * self.width
    def get_perimeter(self):
        return (2 * (self.width + self.length))

class Square(Rectangle):
    def __init__(self,length):
        Rectangle.__init__(self,length,length)


## super()
Return a proxy object that delegates method calls to a parent or sibling class of type. This is useful for accessing inherited methods that have been overridden in a class

In [5]:
class Shape:
    def __init__(self,side):
        self.side = side
    def num_side(self):
        return self.side
    def get_area(self):
        raise NotImplementedError
    def get_perimeter(self):
        raise NotImplementedError
    

class Rectangle(Shape):
    def __init__(self,length,width):
        super().__init__(2)
        self.length = length
        self.width = width
    def get_area(self):
        return self.length * self.width
    def get_perimeter(self):
        return (2 * (self.width + self.length))

class Square(Rectangle):
    def __init__(self,length):
        super().__init__(length,length)

Method Resolution Order (mro)

In [15]:
Square.__mro__, Rectangle.__mro__, Shape.__mro__

((__main__.Square, __main__.Rectangle, __main__.Shape, object),
 (__main__.Rectangle, __main__.Shape, object),
 (__main__.Shape, object))

In [21]:
class Employee:
    def __init__(self,id,name):
        self.id = id
        self.name = name

class ContractEmployee(Employee):
    def __init__(self,id,name,base_salary):
        super(ContractEmployee,self).__init__(id,name)      # equal to super().__init__(id,name)
        self.base_salary = int(base_salary)
    def pay(self):
        return int(self.base_salary)
    def bonus(self,percentage=0.5):
        return int(self.base_salary * (1 + percentage))
    def increase_salary(self,percentage=0.2):
        self.base_salary = int(self.base_salary * (1 + percentage))

class Manager(ContractEmployee):
    def __init__(self,id,name,base_salary = 10_000_000):
        super(Manager,self).__init__(id,name,base_salary)
    def pay(self,comission=1_000_000):
        return int(super().pay() + comission)                  # augmenting method with super()

## Multiple Inheritance

In [43]:
class A:
    def __init__(self):
        print('A')
    def fn(self):
        print('fn a')

class B:
    def __init__(self):
        print('B')
    def fn(self):
        print('fn b')

class C(A,B):
    def __init__(self):
        super().__init__()      # by default super() will return the class based on the inheritance order or by mro
        print('C')
    def fn(self):
        print('fn C')
        super().fn()

class D(B,A):
    def __init__(self):
        super().__init__()
        print('D')
    def fn(self):
        print('fn D')
        super().fn()

C.__mro__, D.__mro__

((__main__.C, __main__.A, __main__.B, object),
 (__main__.D, __main__.B, __main__.A, object))

In [40]:
obj = C();obj.fn()

B
C
fn C
fn b


In [39]:
obj = D();obj.fn()

A
D
fn D
fn a


In [42]:
class C(A,B):
    def __init__(self):
        B.__init__(self)            
        print('C')
    def fn(self):
        print('fn C')
        B.fn(self)

class D(B,A):
    def __init__(self):
        A.__init__(self)
        print('D')
    def fn(self):
        print('fn D')
        A.fn(self)

In [37]:
obj = C(); obj.fn()

B
C
fn C
fn b


In [41]:
obj = D();obj.fn()

A
D
fn D
fn a


## Abstract Base Class
Abstract classes are classes that contain one or more abstract methods. An abstract method is a method that is declared, but contains no implementation. Abstract classes cannot be instantiated, and require subclasses to provide implementations for the abstract methods

In [84]:
from abc import ABCMeta, abstractmethod

In [92]:
class A(metaclass=ABCMeta):
    @abstractmethod
    def foo(self,):
        pass

class B(A):
    def foo(self,value):
        print(value)

In [91]:
a = A()

TypeError: Can't instantiate abstract class A with abstract methods foo

In [94]:
b = B()
b.foo(10)

10


In [77]:
import numpy as np


class File(ABC):
    def __init__(self,fid):
        self.fid = fid
    @abstractmethod
    def read_content(self):
        pass
    @abstractmethod
    def write_content(self):
        pass
    @abstractmethod
    def clear_content(self):
        pass
    

class TransparentFile(File):
    def __init__(self,fid,size):
        super().__init__(fid)
        self.size = size
        self.data = np.zeros(self.size)
    def read_content(self,offset,length):
        return self.data[offset:offset+length]
    def write_content(self,offset,data):
        if len(data) > (self.size - offset): raise IndexError
        if not isinstance(data,np.ndarray): data = np.array(data)
        self.data[offset : offset + len(data)] = data
    def clear_content(self):
        self.data = []
        

In [78]:
a = TransparentFile('0101',100)
a.write_content(0,[1,2,3,4,5])
a.read_content(0,5)

array([1., 2., 3., 4., 5.])

## Composites
A composite class contains an object of another class known to as component. A composite class has a `has a` relationship with its component class

In [46]:
class Person:
    def __init__(self,name,age,address,gender):
        self.name = name
        self.age = age
        self.address = address
        self.gender = gender
    def __str__(self):
        return f'Name: {self.name}, Age: {self.age}, Address: {self.address}, Gender: {self.gender}'

class Employee:
    def __init__(self,id,person):
        self.person = person
    def __getattr__(self,attr):         # magic method to attach person instance attribute to Employee instance as convenient
        return getattr(self.person,attr)

In [50]:
alice = Person('Alice',22,'Jane street','Female')
worker1 = Employee(1,alice)

worker1.name,worker1.age,worker1.gender # directly access person instance attribute in Employee object due to __getattr__

('Alice', 22, 'Female')

In [49]:
print(worker1.person)

Name: Alice, Age: 22, Address: Jane street, Gender: Female


In [52]:
class House:
    def __init__(self, *args):
        self.members = list(args)
    def add_member(self,person):
        self.members.append(person)
    def show_members(self):
        for person in self.members:
            print(person)

In [53]:
bob = Person('Bob',60,'Jane street','Male')
alice = Person('Alice',22,'Jane street','Female')
sue = Person('Sue','57','Jane street','Female')

house = House(bob,alice,sue)
house.show_members()

Name: Bob, Age: 60, Address: Jane street, Gender: Male
Name: Alice, Age: 22, Address: Jane street, Gender: Female
Name: Sue, Age: 57, Address: Jane street, Gender: Female


In [54]:
class Session:
    pass

class Database:
    def __init__(self):
        self.session = Session()

## Class Introspection Tools

In [95]:
class Person:
    def __init__(self,name,job=None,pay=0):
        self.name=name
        self.job=job
        self.pay=int(pay)

In [97]:
bob = Person('Bob Smith')

bob.__class__,bob.__class__.__name__

(__main__.Person, 'Person')

In [98]:
list(bob.__dict__.keys())

['name', 'job', 'pay']

In [99]:
for key in bob.__dict__:
    fmt = "{0}:{1}".format(key,bob.__dict__[key])
    print(fmt)

name:Bob Smith
job:None
pay:0


In [100]:
class Manager(Person):
    def __init__(self,name,pay):
        Person.__init__(self,name,job='mgr',pay = pay)
    def givePayRaise(self,percent,bonus = 0.10):
        Person.givePayRaise(self,percent + bonus)

In [101]:
tom = Manager('Tom Shelby',100000)

Manager.__base__, Manager.__bases__

(__main__.Person, (__main__.Person,))

A Generic Display Tool

In [102]:
class AttrDisplay:
    def gatherAttrs(self):
        attrs = []
        for key in self.__dict__:
            attrs.append("%s=%s" % (key, getattr(self,key)))
        return ', '.join(attrs)
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, self.gatherAttrs())

In [103]:
class Person(AttrDisplay):
    def __init__(self, name, job = None, pay = 0):
        self.name = name
        self.job = job
        self.pay = pay
    def getLastName(self):
        return self.name.split()[-1]
    def givePayRaise(self, percent):
        self.pay = int(self.pay * (1 + percent))

In [107]:
class Manager(Person):
    def __init__(self, name, pay):
        super().__init__(name,'mgr',pay)
    def givePayRaise(self,percent,bonus = 0.10):
        super().givePayRaise((percent + bonus))

In [108]:
bob = Person('Bob Smith')
sue = Person('Sue Jones', job = 'dev', pay = 100000)
print(bob)
print(sue)
print(bob.getLastName(), sue.getLastName())
sue.givePayRaise(.10)
print(sue)
tom = Manager('Tom Jones',50000)
tom.givePayRaise(0.10)
print(tom)

Person(name=Bob Smith, job=None, pay=0)
Person(name=Sue Jones, job=dev, pay=100000)
Smith Jones
Person(name=Sue Jones, job=dev, pay=110000)
Manager(name=Tom Jones, job=mgr, pay=60000)
